In [1]:
import scantelligent.functions as st
from time import sleep
from pynput import keyboard
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from datetime import datetime

Start a session

In [ ]:
session_path, images_path, logfile_path, parameters = st.initialize()

Manual log entries

In [ ]:
message = "This is a manual entry. You can use this to add timestamped comments to the active logfile."
st.logprint(message, logfile = logfile_path)

Simple auto approach

In [ ]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, land_tip = True, timeout = 1000, z_voltage = 220, adjust_percentile = 60)

Simple coarse move in any direction

In [ ]:
st.move_over(direction = "north", steps = 500, z_steps = 10, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 220, velocity_threshold = .02)
st.scan_control(action = "start", scan_direction = "down")

In [ ]:
st.auto_approach(I_approach = 100, p_gain_approach = 30, t_const_approach = 167, land_tip = True, timeout = 1000, z_voltage = 220, adjust_percentile = 60)
st.scan_control(action = "start", scan_direction = "down")

Some scanning, coarse moving, changing parameters

In [ ]:
st.scan_control(action = "start", scan_direction = "up")
st.move_over(direction = "north", steps = 20, z_steps = 10, I_approach = 100, p_gain_approach = 30, t_const_approach = 167, xy_voltage = 246, z_voltage = 220, velocity_threshold = .02, adjust_percentile = 60)
st.change_bias(-2)
st.change_scan_window(center = [30, -100], size = 120, grid = 260, angle = 22)
st.scan_control(action = "start", scan_direction = "down")

In [ ]:
st.scan_control(action = "start", scan_direction = "down", velocity_threshold = .2)

In [ ]:
st.scan_control(action = "stop")

Change functions. They return the old parameters as tuples. Any parameters that are left None or are omitted will remain unchanged.

In [ ]:
V_old = st.change_bias(V = None)
I_old, p_gain_old, t_const_old = st.change_feedback(I = None, p_gain = None, t_const = None)
center_old, size_old, grid_old, angle_old = st.change_scan_window(size = None, grid = None, angle = None)

Tip tracker. Make it time out at 20 seconds. Then return and visualize the x(t), y(t) and z(t) trajectories, with one data point generated every 0.5 seconds. Meanwhile, calculate the surface roughness every 2 seconds.

In [ ]:
txyz = st.tip_tracker(sampling_time = .5, exit_when_still = False, timeout = 20, tracking_info = False, monitor_roughness = True, measurement_interval = 2)
plt.plot(txyz[1], txyz[2])
#plt.plot(txyz[0], txyz[2])
#plt.plot(txyz[0], txyz[3])
plt.show()

Image analysis

In [ ]:
image = st.get_latest_scan(preferred_channels = ["Z", "LI Demod 1 X (A)"], scan_direction = "forward", subtraction = "plane", clip_fraction = .01)
plt.imshow(image, cmap = "gray")
plt.show()

In [3]:
session_path = st.get_session_path()
file_data = st.get_file_data(session_path = session_path)
sxm_files = file_data.image_files
scan_data = st.get_scan(sxm_files[3])
z_img = scan_data.scans[0, 0]

In [ ]:
clip_values = st.clip_range(z_img, method = "percentiles", values = [.2, .8])
histogram = st.get_image_statistics(z_img).histogram
z_img_b = st.background_subtract(z_img)
clip_values_b = st.clip_range(z_img_b, method = "percentiles", values = [.2, .8])
histogram_b = st.get_image_statistics(z_img_b).histogram

fig, ax = plt.subplots(1, 3)
ax[0].imshow(z_img, vmin = clip_values[0], vmax = clip_values[1], cmap = "gray")
ax[1].plot(-histogram[1], histogram[0] - np.mean(histogram[0]))
ax[1].plot(histogram_b[1], histogram_b[0])
ax[2].imshow(z_img_b, vmin = clip_values_b[0], vmax = clip_values_b[1], cmap = "gray")
fig.set_size_inches(10, 5)
plt.show()